In [12]:
import os; os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [13]:
from tqdm.auto import tqdm

from pathlib import Path
import numpy as np
import pandas as pd
import scipy
from pickle import dump, load
import matplotlib.pyplot as plt
import cv2
import random
from IPython.display import clear_output
import tensorflow as tf
# import plotly.express as px
from tensorflow.keras import layers
from functools import partial
from PIL import Image
import re

import torch
import piq

# Datos

In [14]:
(Xtrain, Ytrain), (Xtest, Ytest) = tf.keras.datasets.mnist.load_data()

Xtrain = np.expand_dims(Xtrain,-1)
Xtest = np.expand_dims(Xtest,-1)

In [17]:
def var_pos(X, y, desp_h, desp_v):
  X_big = np.zeros((X.shape[0],256,256,1))
  X_big[:,114+desp_v:114+28+desp_v,114+desp_h:114+28+desp_h] = X[:,:,:]
  return X_big.astype(np.float32), y.astype(np.int32)

def _fixup_shape(images, labels):
    images.set_shape((256,256,256,1))
    labels.set_shape((256,))
    return images, labels

dst_train = tf.data.Dataset.from_tensor_slices((Xtest, Ytest)).batch(256, drop_remainder=True)

var_pos_loquesea = partial(var_pos,desp_h = 0, desp_v = 0)

def tf_function(x,y):
  return tf.numpy_function(var_pos_loquesea, [x,y], (tf.float32, tf.int32))


dst_big = dst_train.map(tf_function).map(_fixup_shape)

Xtrain_big_VGG = dst_big.map(lambda x,y: (tf.keras.applications.vgg16.preprocess_input(
    tf.image.grayscale_to_rgb(tf.convert_to_tensor(x)), data_format=None
),y) )

# VGG

In [18]:
VGG16 = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(256,256,3),
)

for capa in VGG16.layers:
    capa.trainable = False

    model_VGG16 = tf.keras.Sequential([
    VGG16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation = "softmax")
])

model_VGG16.compile(optimizer = "adam", metrics=["accuracy"], loss = "sparse_categorical_crossentropy")
history = model_VGG16.fit(Xtrain_big_VGG, epochs = 10)

Epoch 1/10
39/39 [==============================] - 221s 6s/step - loss: 2.1710 - accuracy: 0.6291
Epoch 2/10
39/39 [==============================] - 220s 6s/step - loss: 0.3060 - accuracy: 0.9078
Epoch 3/10
39/39 [==============================] - 219s 6s/step - loss: 0.1717 - accuracy: 0.9475
Epoch 4/10
39/39 [==============================] - 219s 6s/step - loss: 0.1239 - accuracy: 0.9628
Epoch 5/10
39/39 [==============================] - 220s 6s/step - loss: 0.1004 - accuracy: 0.9710
Epoch 6/10
39/39 [==============================] - 216s 5s/step - loss: 0.0846 - accuracy: 0.9761
Epoch 7/10
39/39 [==============================] - 216s 5s/step - loss: 0.0728 - accuracy: 0.9798
Epoch 8/10
39/39 [==============================] - 218s 6s/step - loss: 0.0632 - accuracy: 0.9837
Epoch 9/10
39/39 [==============================] - 218s 6s/step - loss: 0.0549 - accuracy: 0.9865
Epoch 10/10
39/39 [==============================] - 218s 6s/step - loss: 0.0480 - accuracy: 0.9896


In [19]:
model_VGG16.save('./model_VGG16.h5')

# ResNet50

In [20]:
Xtrain_big_RN = dst_big.map(lambda x,y: (tf.keras.applications.resnet50.preprocess_input(
    tf.image.grayscale_to_rgb(tf.convert_to_tensor(x)), data_format=None),y) )

ResNet50 = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(256,256,3)
)

for capa in ResNet50.layers:
    capa.trainable = False

model_ResNet50 = tf.keras.Sequential([
    ResNet50,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10, activation = "softmax")
])

model_ResNet50.compile(optimizer = "adam", metrics=["accuracy"], loss = "sparse_categorical_crossentropy")

history = model_ResNet50.fit(Xtrain_big_RN, epochs = 10)

Epoch 1/10
39/39 [==============================] - 212s 5s/step - loss: 1.5530 - accuracy: 0.5291
Epoch 2/10
39/39 [==============================] - 209s 5s/step - loss: 0.9053 - accuracy: 0.7481
Epoch 3/10
39/39 [==============================] - 213s 5s/step - loss: 0.7398 - accuracy: 0.7911
Epoch 4/10
39/39 [==============================] - 214s 5s/step - loss: 0.6520 - accuracy: 0.8127
Epoch 5/10
39/39 [==============================] - 214s 5s/step - loss: 0.5948 - accuracy: 0.8292
Epoch 6/10
39/39 [==============================] - 215s 5s/step - loss: 0.5529 - accuracy: 0.8382
Epoch 7/10
39/39 [==============================] - 213s 5s/step - loss: 0.5204 - accuracy: 0.8482
Epoch 8/10
39/39 [==============================] - 212s 5s/step - loss: 0.4940 - accuracy: 0.8536
Epoch 9/10
39/39 [==============================] - 211s 5s/step - loss: 0.4722 - accuracy: 0.8623
Epoch 10/10
39/39 [==============================] - 213s 5s/step - loss: 0.4536 - accuracy: 0.8669


In [21]:
model_ResNet50.save('./model_ResNet50.h5')